In [1]:
import os
%pwd
%cd ..
%pwd

c:\Users\juanp\OneDrive\Escritorio\FlyingFree\WineQualityMLFlow


'c:\\Users\\juanp\\OneDrive\\Escritorio\\FlyingFree\\WineQualityMLFlow'

In [4]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/juanpegarcia11/WineQualityMLFlow.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="juanpegarcia11"
os.environ["MLFLOW_TRACKING_PASSWORD"]="f23bd58860909feaf183020ef3f43ad3c1cef4e7"

In [15]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir : Path
    test_data_path : Path
    model_path : Path
    metric_file_name: Path
    all_params: dict
    target_column: str
    mlflow_uri: str

In [16]:
from WineQualityMLFlow.constants import *
from WineQualityMLFlow.utils.common import read_yaml, create_directories

In [32]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])
    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN


        model_trainer_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            metric_file_name=config.metric_file_name,
            all_params=params,
            target_column=schema.name,
            mlflow_uri="https://dagshub.com/juanpegarcia11/WineQualityMLFlow.mlflow"
        )

        return(model_trainer_config)

In [18]:
import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib
#from WineQualityMLFlow.entity.config_entity import ModelEvaluationConfig
from WineQualityMLFlow.utils.common import save_json
from pathlib import Path

In [30]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
        create_directories([self.config.root_dir])
    


    def eval_metrics(self, actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    
    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]

        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme

        with mlflow.start_run():
            
            # Predict and evaluate model metrics
            predicted_qualities = model.predict(test_x)
            rmse, mae, r2 = self.eval_metrics(test_y, predicted_qualities)

            # Saving metrics as local
            scores = {
                "rmse": rmse, 
                "mae": mae,
                "r2": r2
            }
            print(scores)
            save_json(path=Path(self.config.metric_file_name), data= scores)

            # Logging params and metrics into MLFlow
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(scores)

            # Model Registry
            if tracking_url_type_store != "file":
                mlflow.sklearn.log_model(model, "model", registered_model_name="ElasticNetModel")
            else:
                mlflow.sklearn.log_model(model,"model")

In [33]:
try:
    config = ConfigurationManager()
    model_eval_config = config.get_model_evaluation_config()
    obj = ModelEvaluation(model_eval_config)
    obj.log_into_mlflow()
except Exception as e:
    raise e


[2023-08-14 14:07:02,351: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-14 14:07:02,355: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-14 14:07:02,361: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-08-14 14:07:02,365: INFO: common: created directory at: artifacts]
[2023-08-14 14:07:02,367: INFO: common: created directory at: artifacts/model_evaluation]
{'rmse': 0.7520962985199128, 'mae': 0.6163817639565289, 'r2': 0.0779124536726411}
[2023-08-14 14:07:02,805: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


Registered model 'ElasticNetModel' already exists. Creating a new version of this model...
2023/08/14 14:07:23 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: ElasticNetModel, version 3
Created version '3' of model 'ElasticNetModel'.
